In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
import json

In [3]:
with open('../temp_files/route_dict_journal.json', 'r') as json_file:
    route_dict = json.load(json_file)

In [4]:
route_dict['1']['link_idx_list']

[92, 81, 84, 112]

In [5]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

### create route_choice_probability_matrix $\bf{P} = \left[p_{ir}\right]$ (\# of OD pairs $\times$ \# of routes)

In [6]:
od_nodes_dict = zload('../temp_files/od_nodes_dict_journal.pkz')

In [7]:
od_nodes_dict

{0: {1},
 1: {2, 4, 5},
 2: {3},
 3: {6, 8},
 4: {7, 9},
 5: {10, 11},
 6: {12, 15},
 7: {13, 14},
 8: {16, 17, 18, 19},
 9: {20},
 10: {21},
 11: {22},
 12: {23, 24, 25, 26, 27, 28},
 13: {29},
 14: {30, 31, 32, 33, 34},
 15: {35, 71},
 16: {36},
 17: {37, 38, 39, 40, 41},
 18: {42},
 19: {43, 44, 45, 46, 47},
 20: {48, 74},
 21: {49, 73},
 22: {50, 51},
 23: {52},
 24: {53},
 25: {54, 68, 70},
 26: {55, 56},
 27: {57},
 28: {58},
 29: {59, 60, 72},
 30: {61},
 31: {62, 63, 64, 65, 66},
 32: {67},
 33: {69}}

In [8]:
# od_pair_label_dict = {}

# idx = 0
# for origi in range(len(od_nodes_dict)):
#     for desti in range(len(od_nodes_dict)):
#         if origi != desti:
#             od_pair_label_dict[idx] = [origi, desti]
#             idx += 1
            
# zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [9]:
od_list = []
od_list.append(11) # add Boston downtown
for key in od_nodes_dict.keys():
    if len(od_nodes_dict[key]) > 2:
        od_list.append(key)

In [10]:
od_list

[11, 1, 8, 12, 14, 17, 19, 25, 29, 31]

In [11]:
# od_pair_label_dict = {}

# idx = 0
# for origi in range(len(od_nodes_dict)):
#     for desti in range(len(od_nodes_dict)):
#         if origi != desti:
#             od_pair_label_dict[idx] = [origi, desti]
#             idx += 1
            
# zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [12]:
od_pair_label_dict = {}

idx = 0
for origi in od_list:
    for desti in od_list:
        if origi != desti:
            od_pair_label_dict[idx] = [origi, desti]
            idx += 1
            
zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [13]:
od_pair_label_dict

{0: [11, 1],
 1: [11, 8],
 2: [11, 12],
 3: [11, 14],
 4: [11, 17],
 5: [11, 19],
 6: [11, 25],
 7: [11, 29],
 8: [11, 31],
 9: [1, 11],
 10: [1, 8],
 11: [1, 12],
 12: [1, 14],
 13: [1, 17],
 14: [1, 19],
 15: [1, 25],
 16: [1, 29],
 17: [1, 31],
 18: [8, 11],
 19: [8, 1],
 20: [8, 12],
 21: [8, 14],
 22: [8, 17],
 23: [8, 19],
 24: [8, 25],
 25: [8, 29],
 26: [8, 31],
 27: [12, 11],
 28: [12, 1],
 29: [12, 8],
 30: [12, 14],
 31: [12, 17],
 32: [12, 19],
 33: [12, 25],
 34: [12, 29],
 35: [12, 31],
 36: [14, 11],
 37: [14, 1],
 38: [14, 8],
 39: [14, 12],
 40: [14, 17],
 41: [14, 19],
 42: [14, 25],
 43: [14, 29],
 44: [14, 31],
 45: [17, 11],
 46: [17, 1],
 47: [17, 8],
 48: [17, 12],
 49: [17, 14],
 50: [17, 19],
 51: [17, 25],
 52: [17, 29],
 53: [17, 31],
 54: [19, 11],
 55: [19, 1],
 56: [19, 8],
 57: [19, 12],
 58: [19, 14],
 59: [19, 17],
 60: [19, 25],
 61: [19, 29],
 62: [19, 31],
 63: [25, 11],
 64: [25, 1],
 65: [25, 8],
 66: [25, 12],
 67: [25, 14],
 68: [25, 17],
 69: [2

In [14]:
# number of OD pairs
number_of_od_pairs = len(od_pair_label_dict)

In [15]:
od_pair_label_dict[0]

[11, 1]

In [16]:
od_pair_routes_dict = {}
for row in range(number_of_od_pairs):
    routes = []
    for column in range(len(route_dict)):
        if route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            routes.append(column)
            
    route_length_list = [route_dict[str(column)]['length_of_route'] for column in routes]
    
    refined_routes = []

    number_of_routes_per_OD_pair = 10
    if len(routes) <= number_of_routes_per_OD_pair:
        refined_routes = routes
    else:
        refined_idx_list = [np.argsort(route_length_list)[i] for i in range(number_of_routes_per_OD_pair)]
        refined_routes = [routes[idx] for idx in refined_idx_list]
        
    od_pair_routes_dict[row] = refined_routes

In [17]:
od_pair_routes_dict

{0: [4809, 4810, 4811, 5361, 5362, 5363, 10295, 10296, 10297],
 1: [1909, 1912, 1913, 2937, 1910, 2938, 2940, 1911, 2939, 2941],
 2: [2934, 2931, 9083, 2955, 2958, 2959, 2956, 3006, 9084, 2960],
 3: [15475, 15476, 15469, 15477, 14709, 15478, 15472, 15473, 15479, 15474],
 4: [5418, 5427, 5442, 5428, 5443, 5419, 15442, 13571, 15443, 13572],
 5: [5481, 5482, 5483, 7587, 7588, 5475, 7589, 6312, 5476, 6313],
 6: [2744, 2745, 2746, 6052, 6053, 6054, 7141, 7142, 7143],
 7: [2750, 2751, 2752, 13511, 13512, 13513, 13520, 13521, 13522],
 8: [12555, 13502, 12556, 13503, 13504, 12557, 7129, 12495, 12496, 7130],
 9: [577, 578, 579, 4268, 4269, 4270, 6783, 6784, 6785],
 10: [4271, 4292, 4293, 4294, 4272, 4273, 10944, 4277, 10947, 4278],
 11: [1313, 1314, 1315, 1310, 1311, 1298, 1316, 1319, 1312, 1320],
 12: [6348, 2877, 6349, 1304, 1305, 2333, 2334, 2878, 2335, 1331],
 13: [14757, 14758, 2342, 14759, 2343, 2339, 2344, 2340, 2341, 2345],
 14: [10489, 10490, 10491, 14691, 14692, 14693, 14694, 14688, 1

In [18]:
# re-label routes
new_route_label_dict = {}
route_idx = 0
for od_pair_idx in range(len(od_pair_routes_dict)):
    for route in od_pair_routes_dict[od_pair_idx]:
        new_route_label_dict[route_idx] = route
        route_idx += 1

In [19]:
new_route_label_dict

{0: 4809,
 1: 4810,
 2: 4811,
 3: 5361,
 4: 5362,
 5: 5363,
 6: 10295,
 7: 10296,
 8: 10297,
 9: 1909,
 10: 1912,
 11: 1913,
 12: 2937,
 13: 1910,
 14: 2938,
 15: 2940,
 16: 1911,
 17: 2939,
 18: 2941,
 19: 2934,
 20: 2931,
 21: 9083,
 22: 2955,
 23: 2958,
 24: 2959,
 25: 2956,
 26: 3006,
 27: 9084,
 28: 2960,
 29: 15475,
 30: 15476,
 31: 15469,
 32: 15477,
 33: 14709,
 34: 15478,
 35: 15472,
 36: 15473,
 37: 15479,
 38: 15474,
 39: 5418,
 40: 5427,
 41: 5442,
 42: 5428,
 43: 5443,
 44: 5419,
 45: 15442,
 46: 13571,
 47: 15443,
 48: 13572,
 49: 5481,
 50: 5482,
 51: 5483,
 52: 7587,
 53: 7588,
 54: 5475,
 55: 7589,
 56: 6312,
 57: 5476,
 58: 6313,
 59: 2744,
 60: 2745,
 61: 2746,
 62: 6052,
 63: 6053,
 64: 6054,
 65: 7141,
 66: 7142,
 67: 7143,
 68: 2750,
 69: 2751,
 70: 2752,
 71: 13511,
 72: 13512,
 73: 13513,
 74: 13520,
 75: 13521,
 76: 13522,
 77: 12555,
 78: 13502,
 79: 12556,
 80: 13503,
 81: 13504,
 82: 12557,
 83: 7129,
 84: 12495,
 85: 12496,
 86: 7130,
 87: 577,
 88: 578,
 8

In [20]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

In [21]:
new_route_dict = {}
for key in new_route_label_dict.keys():
    new_route_dict[key] = route_dict[str(new_route_label_dict[key])]

In [22]:
new_route_dict

{0: {u'desti': 1,
  u'length_of_route': 20.61412691556436,
  u'link_idx_list': [59, 54, 25, 13],
  u'node_list': [22, 16, 17, 6, 4],
  u'origi': 11},
 1: {u'desti': 1,
  u'length_of_route': 21.150250340488512,
  u'link_idx_list': [61, 25, 13],
  u'node_list': [22, 17, 6, 4],
  u'origi': 11},
 2: {u'desti': 1,
  u'length_of_route': 22.13084965216668,
  u'link_idx_list': [59, 33, 21, 13],
  u'node_list': [22, 16, 8, 6, 4],
  u'origi': 11},
 3: {u'desti': 1,
  u'length_of_route': 28.527571308662345,
  u'link_idx_list': [59, 54, 25, 9, 7],
  u'node_list': [22, 16, 17, 6, 3, 2],
  u'origi': 11},
 4: {u'desti': 1,
  u'length_of_route': 29.063694733586498,
  u'link_idx_list': [61, 25, 9, 7],
  u'node_list': [22, 17, 6, 3, 2],
  u'origi': 11},
 5: {u'desti': 1,
  u'length_of_route': 30.044294045264667,
  u'link_idx_list': [59, 33, 21, 9, 7],
  u'node_list': [22, 16, 8, 6, 3, 2],
  u'origi': 11},
 6: {u'desti': 1,
  u'length_of_route': 29.5922932335393,
  u'link_idx_list': [67, 45, 19],
  u'nod

In [23]:
with open('../temp_files/new_route_dict_journal.json', 'w') as json_file:
    json.dump(new_route_dict, json_file)

In [24]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)

In [25]:
# number of routes
number_of_routes = len(new_route_dict)

In [26]:
number_of_routes

894

In [27]:
# initialize the route_choice_probability_matrix P
P = np.zeros((number_of_od_pairs, number_of_routes))

np.shape(P)

(90, 894)

In [28]:
for row in range(number_of_od_pairs):
    for column in range(number_of_routes):
        if new_route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        new_route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            P[row, column] = 1

In [29]:
# for column in range(r):
#     assert(sum(P[:, column]) <= 1)

In [30]:
# for row in range(i):
#     assert(sum(P[row, :]) >= 1)

### Use sparse matrix to save memory

In [31]:
P = csr_matrix(P)

In [32]:
zdump(P, '../temp_files/OD_pair_route_incidence_journal.pkz')

In [33]:
link_label_dict = zload('../temp_files/link_label_dict_journal.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_journal_.pkz')

In [34]:
len(link_label_dict)

258

In [35]:
# number of links
number_of_links = len(link_label_dict)

In [36]:
number_of_links

258

In [37]:
# initialize the link_route incidence matrix
A = np.zeros((number_of_links, number_of_routes))

# create link_route incidence matrix A
# route index
for i in range(number_of_routes):
    # link index
    for j in range(number_of_links):
        if j in new_route_dict[str(i)]['link_idx_list']:
            A[j, i] = 1

### Use sparse matrix to save memory

In [38]:
A = csc_matrix(A)        

zdump(A, '../temp_files/link_route_incidence_matrix_journal.pkz')

In [39]:
matrix_rank(A.todense())

147

In [40]:
print(np.size(A, 0))
print(np.size(A, 1))

258
894
